## **Распознавание объектов на изображениях с использованием сверточных нейронных сетей (CNN)**
## Автор: Евтых Ратмир  
## Дата: 2025-03-30

In [4]:
'''
Компилятор может пожаловаться,
что такая библиотека уже есть и предложить перезапуск,
в таком случае отказываемся от перезапуска
'''
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-e8xi4ur1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-e8xi4ur1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyyaml
!pip install pycocotools

In [11]:
import os
import cv2
import torch
import json
from collections import defaultdict, Counter
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer


def setup_model():
    # Создаем конфигурацию для модели Mask R-CNN с Feature Pyramid Network (FPN)
    cfg = get_cfg()

    # Загружаем конфигурацию из model zoo Detectron2 для instance segmentation
    cfg.merge_from_file(model_zoo.get_config_file
     (
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
        )
     )

    # Устанавливаем порог уверенности для отображения предсказаний
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

    # Загружаем предобученные веса для данной модели
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
        )

    # Создаем предиктор
    predictor = DefaultPredictor(cfg)

    # Получаем метаданные для отображения имен классов
    metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])

    return predictor, metadata, cfg


def detect_objects(
    image_path,
    output_path,
    predictor,
    metadata,
    ):
    # Загружаем изображение
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(
            f"Не удалось загрузить изображение по пути: {image_path}"
            )

    outputs = predictor(image)
    instances = outputs["instances"]

    # Визуализируем результаты
    v = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.2)
    out = v.draw_instance_predictions(instances.to("cpu"))
    result_image = out.get_image()[:, :, ::-1]

    # Сохраняем изображение с нарисованными сегментациями
    cv2.imwrite(output_path, result_image)
    print(f"Результат сохранён в {output_path}")

    # Подсчет обьектов
    if instances.has("pred_classes"):
      pred_classes = instances.pred_classes.cpu().numpy()
      counts = Counter(pred_classes)

      # Сопоставляем числовые метки с именами классов через metadata
      class_names = metadata.get("thing_classes", None)
      if class_names:
          print("Обнаруженные объекты:")
          for cls_id, cnt in counts.items():
              print(f"{class_names[int(cls_id)]}: {cnt}")
      else:
          print("Обнаруженные классы (числовые метки):", counts)
    else:
        print("Объекты не обнаружены.")

def test_on_coco(cfg, predictor):
    register_coco_instances(
        "my_coco_val", {},
        "drive/MyDrive/annotations/instances_val2017.json",
        "drive/MyDrive/val2017/"
    )

    # Создаем COCOEvaluator для набора данных валидации COCO 2017
    evaluator = COCOEvaluator(
        "my_coco_val", cfg, False,
        output_dir="drive/MyDrive/coco_eval"
    )
    val_loader = build_detection_test_loader(cfg, "my_coco_val")
    # Выполняем инференс по всему набору данных и вычисляем метрики
    metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
    print("Метрики на COCO 2017 val:")
    print(metrics)

def process_coco_results(json_path):
    # Загружаем данные из JSON-файла
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Группируем результаты по image_id и category_id, сохраняем список score
    summary = defaultdict(lambda: defaultdict(list))
    for det in data:
        image_id = det["image_id"]
        category_id = det["category_id"]
        score = det["score"]
        summary[image_id][category_id].append(score)
    return summary

def print_summary(summary, category_names=None):
    print("Сводная информация по изображениям:")
    # Словарь для общей агрегации: ключ – cat_id, значение – список всех score по всем изображениям
    overall = {}

    for image_id, cats in summary.items():
        print(f"Image {image_id}:")
        for cat_id, scores in cats.items():
            count = len(scores)
            avg_score = sum(scores) / count if count > 0 else 0
            # Добавляем данные в общий словарь
            if cat_id not in overall:
                overall[cat_id] = []
            overall[cat_id].extend(scores)
            # Если есть сопоставление с именами категорий, выводим имя
            if category_names and cat_id in category_names:
                cat_name = category_names[cat_id]
            else:
                cat_name = f"Category {cat_id}"
            print(f"  {cat_name}: {count} обнаружений, средний score = {avg_score:.3f}")
        print()

    # Вывод общей информации по всем изображениям
    print("Общая сводная информация по всем изображениям:")
    for cat_id, scores in overall.items():
        total_count = len(scores)
        overall_avg = sum(scores) / total_count if total_count > 0 else 0
        if category_names and cat_id in category_names:
            cat_name = category_names[cat_id]
        else:
            cat_name = f"Category {cat_id}"
        print(f"  Модель справилась с угадыванием {cat_name}: {total_count} обнаружений, средний score = {overall_avg:.3f}")

def main():
    # predictor, metadata, cfg = setup_model()
    # detect_objects(image_path, output_path, predictor, metadata)

    # test_on_coco(cfg, predictor)
    coco_category_names = {
      0: "__фон__",
      1: "человек",
      2: "велосипед",
      3: "автомобиль",
      4: "мотоцикл",
      5: "самолет",
      6: "автобус",
      7: "поезд",
      8: "грузовик",
      9: "лодка",
      10: "светофор",
      11: "пожарный гидрант",
      12: "дорожный знак",
      13: "знак остановки",
      14: "паркомат",
      15: "скамейка",
      16: "птица",
      17: "кошка",
      18: "собака",
      19: "лошадь",
      20: "овца",
      21: "корова",
      22: "слон",
      23: "медведь",
      24: "зебра",
      25: "жираф",
      26: "шляпа",
      27: "рюкзак",
      28: "зонт",
      29: "обувь",
      30: "очки",
      31: "сумка",
      32: "галстук",
      33: "чемодан",
      34: "фрисби",
      35: "лыжи",
      36: "сноуборд",
      37: "спортивный мяч",
      38: "воздушный змей",
      39: "бейсбольная бита",
      40: "бейсбольная перчатка",
      41: "скейтборд",
      42: "доска для серфинга",
      43: "теннисная ракетка",
      44: "бутылка",
      45: "тарелка",
      46: "бокал",
      47: "чашка",
      48: "вилка",
      49: "нож",
      50: "ложка",
      51: "миска",
      52: "банан",
      53: "яблоко",
      54: "сэндвич",
      55: "апельсин",
      56: "брокколи",
      57: "морковь",
      58: "хот-дог",
      59: "пицца",
      60: "пончик",
      61: "торт",
      62: "стул",
      63: "диван",
      64: "растение в горшке",
      65: "кровать",
      66: "зеркало",
      67: "обеденный стол",
      68: "окно",
      69: "письменный стол",
      70: "унитаз",
      71: "дверь",
      72: "телевизор",
      73: "ноутбук",
      74: "мышь (компьютерная)",
      75: "пульт",
      76: "клавиатура",
      77: "мобильный телефон",
      78: "микроволновка",
      79: "духовка",
      80: "тостер",
      81: "раковина",
      82: "холодильник",
      83: "блендер",
      84: "книга",
      85: "часы",
      86: "ваза",
      87: "ножницы",
      88: "плюшевый мишка",
      89: "фен",
      90: "зубная щетка",
      91: "расческа"
    }

    summary = process_coco_results("drive/MyDrive/coco_eval/coco_instances_results.json")
    print_summary(summary, category_names=coco_category_names)

if __name__ == "__main__":
    main()


Сводная информация по изображениям:
Image 139:
  человек: 2 обнаружений, средний score = 0.882
  стул: 4 обнаружений, средний score = 0.842
  телевизор: 2 обнаружений, средний score = 0.957
  ваза: 5 обнаружений, средний score = 0.714
  холодильник: 3 обнаружений, средний score = 0.830
  растение в горшке: 1 обнаружений, средний score = 0.833
  обеденный стол: 2 обнаружений, средний score = 0.659
  часы: 1 обнаружений, средний score = 0.713
  бутылка: 1 обнаружений, средний score = 0.664

Image 285:
  медведь: 1 обнаружений, средний score = 0.987

Image 632:
  растение в горшке: 2 обнаружений, средний score = 0.992
  стул: 1 обнаружений, средний score = 0.993
  кровать: 1 обнаружений, средний score = 0.990
  бутылка: 1 обнаружений, средний score = 0.969
  книга: 20 обнаружений, средний score = 0.577

Image 724:
  знак остановки: 2 обнаружений, средний score = 0.841
  грузовик: 1 обнаружений, средний score = 0.931
  автомобиль: 1 обнаружений, средний score = 0.549

Image 776:
  плюшевый

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!nvidia-smi  # Должна появиться таблица с информацией о GPU

Tue Apr  1 16:07:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----